In [1]:
import pandas as pd

import openai
import getpass

import re
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
from langdetect import detect

from collections import Counter

from tqdm.notebook import tqdm
import time
import joblib

# from presidio_analyzer import AnalyzerEngine

pd.set_option('display.max_columns', None)

# data load and cleaning

In [2]:
df = pd.read_csv('2022-rappler-articles-ns.csv')
df.shape

/var/folders/tv/psypbvx97n74gbz8rcln8f8m0000gn/T/ipykernel_29134/1893922539.py:1: DtypeWarning: Columns (30,31,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2022-rappler-articles-ns.csv')


(36215, 60)

In [3]:
df.head()

,id,date,date_gmt,modified,modified_gmt,slug,status,type,link,featured_media,comment_status,ping_status,sticky,template,format,categories,tags,movement,article_type,tier,ab_test_titles,ab_test_featured_images,authorship,guid.rendered,title.rendered,content.rendered,content.protected,excerpt.rendered,excerpt.protected,meta.rappler-three-point-summary,meta.claim_author_type,meta.claim_author_name,meta.claim_reviewed,meta.review_rating,meta.schema_article_type,meta.ep_exclude_from_search,meta.is_sponsored,meta.sponsor,ab_tests.titles.started,ab_tests.titles.start_time,ab_tests.titles.end_time,ab_tests.titles.traffic_percentage,ab_tests.titles.variant_traffic_percentage,ab_tests.titles.paused,ab_tests.featured_images.started,ab_tests.featured_images.start_time,ab_tests.featured_images.end_time,ab_tests.featured_images.traffic_percentage,ab_tests.featured_images.variant_traffic_percentage,ab_tests.featured_images.paused,_links.self,_links.collection,_links.about,_links.replies,_links.version-history,_links.predecessor-version,_links.wp:featuredmedia,_links.wp:attachment,_links.wp:term,_links.curies
0,2453846,2023-07-24T17:28:52,2023-07-24T09:28:52,2023-07-24T17:29:11,2023-07-24T09:29:11,norway-government-ministries-cyber-attack-july...,publish,post,https://www.rappler.com/technology/norway-gove...,2453906,open,closed,False,NaN,standard,"[764, 732, 761]","[1724, 2446]",[],[],[],[],[],[5083],urn:uuid:d0f61b60-7e61-4425-b482-66802ebe69e1,Norway government ministries hit by cyber attack,"\n<p>OSLO, Norway – Twelve Norwegian governmen...",False,<p>Twelve Norwegian government ministries are ...,False,['[]'],NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192528000,1692784528000,35,[],True,False,1690192528000,1692784528000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 1, 'href': 'https://www.rappler.com...","[{'id': 2453883, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re..."
1,2453663,2023-07-24T17:23:58,2023-07-24T09:23:58,2023-07-24T17:24:10,2023-07-24T09:24:10,video-marcos-greets-former-presidents-on-way-s...,publish,post,https://www.rappler.com/nation/video-marcos-gr...,2453691,open,closed,False,template-video.php,standard,[696],"[17075, 20514]",[],[],[],[],[],[4999],urn:uuid:eb85ef48-fc93-4410-a396-39d522e20554,WATCH: Marcos greets 2 former presidents on th...,"\n<p>MANILA, Philippines – President Ferdinand...",False,"<p>As he greeted former presidents, President ...",False,['[]'],NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192528000,1692784528000,35,[],True,False,1690192528000,1692784528000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 8, 'href': 'https://www.rappler.com...","[{'id': 2453893, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re..."
2,2453316,2023-07-24T16:45:38,2023-07-24T08:45:38,2023-07-24T16:45:48,2023-07-24T08:45:48,sona-2023-full-text-transcript-speech-presiden...,publish,post,https://www.rappler.com/nation/sona-2023-full-...,2453747,open,closed,False,NaN,standard,"[697, 696, 20536]","[17075, 18036, 20514, 2517]",[],[],[],[],[],[4999],urn:uuid:31073025-a5d6-4ff8-895b-eea8920b318c,FULL TEXT: President Marcos’ State of the Nati...,\n<p><em>Editor&#8217;s Note: The full text of...,False,<p>President Ferdinand Marcos presents to the ...,False,"['[""President', 'Ferdinand', 'Marcos', 'Jr.', ...",NaN,NaN,

In [4]:
df['date'] = pd.to_datetime(df['date'])
business_df = df[(df['date']>='2023-01-01') & df['link'].str.contains('/business/')].copy()
business_df.shape

(1255, 60)

In [5]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text()
    return cleaned_text

business_df['content.cleaned'] = business_df['content.rendered'].apply(remove_html_tags)

In [6]:
pattern = r'[\t\n]|<[\w "@+#%&,./:;=?_-]+>'
pattern2 = r'<.*https?.*>'
pattern3 = r'<(?:a|span|div|em|strong|mark).*>'
pattern4 = r'!function.*;'
pattern5 = r'\.igframe.*}'

business_df['content.cleaned'] = (
    business_df['content.cleaned']
    .apply(unidecode)
    .apply(lambda x: re.sub(pattern, ' ', x))
    .apply(lambda x: re.sub(pattern2, ' ', x))
    .apply(lambda x: re.sub(pattern3, ' ', x))
    .apply(lambda x: re.sub(pattern4, ' ', x))
    .apply(lambda x: re.sub(pattern5, ' ', x))
    .str.replace('&#8217;', "'")
    .str.replace('&#8220;', '"')
    .str.replace('&#8221;', '"')
)

In [7]:
business_df.head()

,id,date,date_gmt,modified,modified_gmt,slug,status,type,link,featured_media,comment_status,ping_status,sticky,template,format,categories,tags,movement,article_type,tier,ab_test_titles,ab_test_featured_images,authorship,guid.rendered,title.rendered,content.rendered,content.protected,excerpt.rendered,excerpt.protected,meta.rappler-three-point-summary,meta.claim_author_type,meta.claim_author_name,meta.claim_reviewed,meta.review_rating,meta.schema_article_type,meta.ep_exclude_from_search,meta.is_sponsored,meta.sponsor,ab_tests.titles.started,ab_tests.titles.start_time,ab_tests.titles.end_time,ab_tests.titles.traffic_percentage,ab_tests.titles.variant_traffic_percentage,ab_tests.titles.paused,ab_tests.featured_images.started,ab_tests.featured_images.start_time,ab_tests.featured_images.end_time,ab_tests.featured_images.traffic_percentage,ab_tests.featured_images.variant_traffic_percentage,ab_tests.featured_images.paused,_links.self,_links.collection,_links.about,_links.replies,_links.version-history,_links.predecessor-version,_links.wp:featuredmedia,_links.wp:attachment,_links.wp:term,_links.curies,content.cleaned
26,2452896,2023-07-24 12:43:49,2023-07-24T04:43:49,2023-07-24T13:01:14,2023-07-24T05:01:14,list-flights-canceled-delayed-typhoon-egay-jul...,publish,post,https://www.rappler.com/business/list-flights-...,2243073,open,closed,False,NaN,standard,"[621, 624]","[2283, 2487, 2802]",[],[],[],[],[],[6936],urn:uuid:71f84171-debc-4aa0-b232-5166cd1de723,"LIST: Canceled flights due to Typhoon Egay, Ju...","\n<p>MANILA, Philippines – Various airports an...",False,<p>Bookmark this page to see the affected flig...,False,"['[""Typhoon', 'Egay', '(Doksuri)', 'prompts', ...",NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192529000,1692784529000,35,[],True,False,1690192529000,1692784529000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 6, 'href': 'https://www.rappler.com...","[{'id': 2453139, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re...","MANILA, Philippines - Various airports and ai..."
34,2452759,2023-07-24 10:44:42,2023-07-24T02:44:42,2023-07-24T10:44:46,2023-07-24T02:44:46,repower-energy-initial-public-offering-july-24...,publish,post,https://www.rappler.com/business/repower-energ...,2452801,open,closed,False,NaN,standard,"[621, 625, 626]","[2716, 2520, 2266]",[],[],[],[],[],[4952],urn:uuid:3f2e008c-7f8f-4853-ba0f-949344983015,Repower Energy goes public in tepid market,"\n<p>MANILA, Philippines – Repower Energy Deve...",False,<p>Repower Energy Development Corporation is j...,False,['[]'],NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192530000,1692784530000,35,[],True,False,1690192530000,1692784530000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 6, 'href': 'https://www.rappler.com...","[{'id': 2452843, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re...","MANILA, Philippines - Repower Energy Developm..."
111,2449638,2023-07-22 14:42:06,2023-07-22T06:42:06,2023-07-22T14:42:11,2023-07-22T06:42:11,coa-finds-anti-red-tape-authority-complaint-re...,publish,post,https://www.rappler.com/business/coa-finds-ant...,2017041,open,closed,False,NaN,standard,"[621, 622]","[73, 1964]",[],[],[],[],[],[4999],urn:uuid:c48afd24-cc1e-4234-868a-32e835be1e40,Anti-Red Tape Authority at risk of &#8216;erod...,"\n<p>MANILA, Philippines – The C

# ner tagging using gpt

## sample queries

In [ ]:
# openai.api_key = getpass.getpass()

In [ ]:
# system_prompt = """
# You are a media analyst who reads news to identify relevant entities and to track trends, sentiments, and patterns in the media to provide insights into various topics.
# """

# main_prompt = """
# Identify all  personalities mentioned in the following article that are important to note to understand the article's context.

# - List down name of PEOPLE only.
# - In listing the names, do NOT add their titles or labels.
# - If there are more than one relevant personality from the article, list all of them in a semi-colon-separated format. EXAMPLE: Rodrigo Duterte;Leni Robredo
# - If there are no relevant personalities mentioned, return "NONE".
# """

In [ ]:
# sample_1 = business_df.loc[26, 'content.cleaned']
# sample_1

In [ ]:
# response = openai.ChatCompletion.create(
#                 model="gpt-3.5-turbo",   # Use GPT-3.5 Turbo model
#                 messages=[
#                     {"role": "system", "content": system_prompt},
#                     {"role": "user", "content": f"{main_prompt}\n\n{sample_1}"}
#                 ]
#             )

# response

In [ ]:
# sample_2 = business_df.loc[127, 'content.cleaned']
# sample_2

In [ ]:
# response = openai.ChatCompletion.create(
#                 model="gpt-3.5-turbo",   # Use GPT-3.5 Turbo model
#                 messages=[
#                     {"role": "system", "content": system_prompt},
#                     {"role": "user", "content": f"{main_prompt}\n\n{sample_2}"}
#                 ]
#             )

# response

In [ ]:
# response['choices'][0]['message']['content']

## query function

In [8]:
openai.api_key = getpass.getpass()

········


In [9]:
system_prompt = """
You are a media analyst who reads news to identify relevant PEOPLE in an article's context.
"""

main_prompt = """
###TASK###
Identify ALL INDIVIDUALS mentioned in the following article who are important to note for understanding the article's context.

- List down names of PEOPLE only. Do NOT include organizations and locations.
- When listing the names, do NOT add their titles or labels.
- If there are multiple relevant personalities mentioned in the article, list all of them in a semi-colon-separated format. EXAMPLE: Rodrigo Duterte; Leni Robredo.
- If no relevant personalities are mentioned, return NONE.

###ARTICLE###
"""

In [10]:
def gpt_entity_recognition(article, system_prompt, main_prompt):
    response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo", 
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"{main_prompt}\n\n{article}"}
                ]
            )
    
    response = response['choices'][0]['message']['content']
    if response.lower() == 'none':
        return list()
    else:
        return [ent.strip() for ent in response.split(';')]

In [12]:
gpt_entity_recognition(business_df['content.cleaned'].iloc[1], system_prompt, main_prompt)

['Eric Peter Roxas']

# applying entity recognition function to df

In [14]:
# joblib.dump(unique_entities[:370], 'uniq.pkl')

unique_entities = joblib.load('uniq.pkl') #list()
counter = 35

for article in tqdm(business_df['content.cleaned'][370:]):
    try:
        r = gpt_entity_recognition(article, system_prompt, main_prompt)
        unique_entities.append(r)
    except:
        counter += 1
        print(f'Failed queries: {counter}')
        unique_entities.append([])
    
    time.sleep(3)

unique_entities[0:10]

  0%|          | 0/885 [00:00<?, ?it/s]

Failed queries: 36
Failed queries: 37
Failed queries: 38
Failed queries: 39
Failed queries: 40
Failed queries: 41
Failed queries: 42
Failed queries: 43
Failed queries: 44
Failed queries: 45
Failed queries: 46
Failed queries: 47
Failed queries: 48
Failed queries: 49
Failed queries: 50
Failed queries: 51
Failed queries: 52
Failed queries: 53
Failed queries: 54
Failed queries: 55
Failed queries: 56
Failed queries: 57
Failed queries: 58
Failed queries: 59
Failed queries: 60
Failed queries: 61
Failed queries: 62
Failed queries: 63
Failed queries: 64
Failed queries: 65
Failed queries: 66
Failed queries: 67
Failed queries: 68
Failed queries: 69
Failed queries: 70
Failed queries: 71
Failed queries: 72
Failed queries: 73


[[],
 ['Eric Peter Roxas', 'Rodrigo Duterte'],
 [],
 ['Edgar Galvante', 'Rodrigo Duterte'],
 [],
 [],
 ['Edgar Injap Sia II'],
 ['Ben Jeffery', 'Colin Graham', 'Daleep Singh'],
 ['Narendra Modi', 'BV Krishna Rao'],
 ['Tanya Ansaldo-Deakin']]

In [16]:
business_df['unique_entities'] = unique_entities

In [17]:
joblib.dump(business_df, 'business.pkl')

['business.pkl']

In [20]:
business_df.to_csv('business.csv', index=False)

# cleaning entity tags

## inspecting unique entities identified

In [31]:
ent_dict = business_df['unique_entities'].explode().dropna().value_counts().to_dict()

In [32]:
for k, v, in ent_dict.items():
    print(k)

Joe Biden
Janet Yellen
Ferdinand Marcos Jr.
Jerome Powell
Kevin McCarthy
Rodrigo Duterte
Jaime Bautista
Jeremy Hunt
Donald Trump
Jamie Dimon
Kazuo Ueda
Kristalina Georgieva
Rishi Sunak
Vladimir Putin
David Malpass
Gautam Adani
Jes Staley
Arsenio Balisacan
Jeffrey Epstein
Benjamin Diokno
Ajay Banga
Christine Lagarde
Risa Hontiveros
Xi Jinping
Chuck Schumer
Felipe Medalla
Manny Pangilinan
Haruhiko Kuroda
Narendra Modi
Grace Poe
Martin Romualdez
Elizabeth Warren
Dave Calhoun
Tayyip Erdogan
Teofilo Guadiz III
Fumio Kishida
Mitch McConnell
Michael Barr
Colm Kelleher
Emmanuel Macron
Bryan Co
Raphael Bostic
Andrew Bailey
Cesar Chiong
Hakeem Jeffries
Ralph Hamers
Dennis Uy
Alexander Novak
Bernie Sanders
Manuel Tamayo
Patrick McHenry
Leni Robredo
Ranil Wickremesinghe
Tim Ghriskey
Manuel V. Pangilinan
Li Qiang
Juan Miguel Zubiri
Denys Shmyhal
Martin Gruenberg
James Bullard
Ulrich Koerner
Prince Abdulaziz bin Salman
Sergio Ermotti
Volodymyr Zelenskiy
Ralf Rivas
Greg Becker
Liz Truss
Sherwin Gatch

## identifying known author names

In [38]:
import numpy as np

def scrape_author_info(article_url):
    response = requests.get(article_url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        author_element = soup.find('a', class_='post-single__author')
        
        if author_element:
            author_name = author_element.get_text(strip=True)
            return author_name
        else:
            return np.nan
    else:
        return np.nan
    
an_list = list()

for url in tqdm(business_df['link'].tolist()):
    auth_name = scrape_author_info(url)
    an_list.append(auth_name)
    
business_df['author.name'] = an_list
business_df.head()

  0%|          | 0/1255 [00:00<?, ?it/s]

,id,date,date_gmt,modified,modified_gmt,slug,status,type,link,featured_media,comment_status,ping_status,sticky,template,format,categories,tags,movement,article_type,tier,ab_test_titles,ab_test_featured_images,authorship,guid.rendered,title.rendered,content.rendered,content.protected,excerpt.rendered,excerpt.protected,meta.rappler-three-point-summary,meta.claim_author_type,meta.claim_author_name,meta.claim_reviewed,meta.review_rating,meta.schema_article_type,meta.ep_exclude_from_search,meta.is_sponsored,meta.sponsor,ab_tests.titles.started,ab_tests.titles.start_time,ab_tests.titles.end_time,ab_tests.titles.traffic_percentage,ab_tests.titles.variant_traffic_percentage,ab_tests.titles.paused,ab_tests.featured_images.started,ab_tests.featured_images.start_time,ab_tests.featured_images.end_time,ab_tests.featured_images.traffic_percentage,ab_tests.featured_images.variant_traffic_percentage,ab_tests.featured_images.paused,_links.self,_links.collection,_links.about,_links.replies,_links.version-history,_links.predecessor-version,_links.wp:featuredmedia,_links.wp:attachment,_links.wp:term,_links.curies,content.cleaned,unique_entities,author.name
26,2452896,2023-07-24 12:43:49,2023-07-24T04:43:49,2023-07-24T13:01:14,2023-07-24T05:01:14,list-flights-canceled-delayed-typhoon-egay-jul...,publish,post,https://www.rappler.com/business/list-flights-...,2243073,open,closed,False,NaN,standard,"[621, 624]","[2283, 2487, 2802]",[],[],[],[],[],[6936],urn:uuid:71f84171-debc-4aa0-b232-5166cd1de723,"LIST: Canceled flights due to Typhoon Egay, Ju...","\n<p>MANILA, Philippines – Various airports an...",False,<p>Bookmark this page to see the affected flig...,False,"['[""Typhoon', 'Egay', '(Doksuri)', 'prompts', ...",NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192529000,1692784529000,35,[],True,False,1690192529000,1692784529000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 6, 'href': 'https://www.rappler.com...","[{'id': 2453139, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re...","MANILA, Philippines - Various airports and ai...",[],Lance Spencer Yu
34,2452759,2023-07-24 10:44:42,2023-07-24T02:44:42,2023-07-24T10:44:46,2023-07-24T02:44:46,repower-energy-initial-public-offering-july-24...,publish,post,https://www.rappler.com/business/repower-energ...,2452801,open,closed,False,NaN,standard,"[621, 625, 626]","[2716, 2520, 2266]",[],[],[],[],[],[4952],urn:uuid:3f2e008c-7f8f-4853-ba0f-949344983015,Repower Energy goes public in tepid market,"\n<p>MANILA, Philippines – Repower Energy Deve...",False,<p>Repower Energy Development Corporation is j...,False,['[]'],NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192530000,1692784530000,35,[],True,False,1690192530000,1692784530000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 6, 'href': 'https://www.rappler.com...","[{'id': 2452843, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re...","MANILA, Philippines - Repower Energy Developm...","[Eric Peter Roxas, Rodrigo Duterte]",Ralf Rivas
111,2449638,2023-07-22 14:42:06,2023-07-22T06:42:06,2023-07-22T14:42:11,2023-07-22T06:42:11,coa-finds-anti-red-tape-authority-complaint-re...,publish,post,https://www.rappler.com/business/coa-finds-ant...,2017041,open,closed,False,NaN,standard,"[621, 622]","[73, 1964]",[],[],[],[],[],[4999],urn:uuid:c48afd24-cc1e-4234-868a

In [39]:
known_authors = list(set(['James Patrick Cruz', 'Lance Spencer Yu', 'Dennis Abrina',
                'Lance Yu', 'Martha Teodoro', 'Jodesz Gavilan', 'Joann Manabat',
                'Chris Burnet Ramos', 'Michelle Abad', 'Lorenz Pasion', 
                'Ralf Rivas', 'Bea Cupin', 'Bonz Magsambol', 'Lian Buan',
                'Sofia Tomacruz', 'Eirenne Lumasang', 'Ryan Macasero'] +
                 business_df['author.name'].dropna().unique().tolist()))

known_authors = known_authors + [f'{n}/Rappler.com' for n in known_authors]
known_authors

['Chris Burnet Ramos',
 'Joann Manabat',
 'Isagani de Castro Jr.',
 'Sofia Tomacruz',
 'Eirenne Lumasang',
 'Martha Teodoro',
 'Dennis Abrina',
 'Lance Spencer Yu',
 'Jodesz Gavilan',
 'Jairo Bolledo',
 'Ralf Rivas',
 'James Patrick Cruz',
 'Bea Cupin',
 'Lian Buan',
 'Lorenz Pasion',
 'Ryan Macasero',
 'Michelle Abad',
 'Bonz Magsambol',
 'Iya Gozum',
 'Lance Yu',
 'Chris Burnet Ramos/Rappler.com',
 'Joann Manabat/Rappler.com',
 'Isagani de Castro Jr./Rappler.com',
 'Sofia Tomacruz/Rappler.com',
 'Eirenne Lumasang/Rappler.com',
 'Martha Teodoro/Rappler.com',
 'Dennis Abrina/Rappler.com',
 'Lance Spencer Yu/Rappler.com',
 'Jodesz Gavilan/Rappler.com',
 'Jairo Bolledo/Rappler.com',
 'Ralf Rivas/Rappler.com',
 'James Patrick Cruz/Rappler.com',
 'Bea Cupin/Rappler.com',
 'Lian Buan/Rappler.com',
 'Lorenz Pasion/Rappler.com',
 'Ryan Macasero/Rappler.com',
 'Michelle Abad/Rappler.com',
 'Bonz Magsambol/Rappler.com',
 'Iya Gozum/Rappler.com',
 'Lance Yu/Rappler.com']

## removing irrelevant entities

In [80]:
ent_err_fmt = {x: [i.replace('-','').strip() for i in x.split('\n')] for x in business_df['unique_entities'].explode() if '\n' in str(x)}
ent_err_fmt_add = {x: [i.replace('-','').strip() for i in x.split(',')] for x in business_df['unique_entities'].explode() if ',' in str(x)}

ent_err_fmt.update(ent_err_fmt_add)
ent_err_fmt

{'Forbes\nCIMB Bank\nPhilippine National Bank (PNB)\nBank of the Philippine Islands (BPI)\nUnion Bank of the Philippines\nBDO Unibank\nLand Bank of the Philippines\nPhilippine Savings Bank (PSBank)\nRizal Commercial Banking Corporation (RCBC)\nMaya Bank\nRobinsons Bank\nTan-owned PNB\nAyala-led BPI\nSy-owned BDO': ['Forbes',
  'CIMB Bank',
  'Philippine National Bank (PNB)',
  'Bank of the Philippine Islands (BPI)',
  'Union Bank of the Philippines',
  'BDO Unibank',
  'Land Bank of the Philippines',
  'Philippine Savings Bank (PSBank)',
  'Rizal Commercial Banking Corporation (RCBC)',
  'Maya Bank',
  'Robinsons Bank',
  'Tanowned PNB',
  'Ayalaled BPI',
  'Syowned BDO'],
 'Ferdinand Marcos Jr.\nDiokno\nRodrigo Duterte\nYale\nHarvard\nWinnie Monsod\nSonny Africa\nMargarito Teves': ['Ferdinand Marcos Jr.',
  'Diokno',
  'Rodrigo Duterte',
  'Yale',
  'Harvard',
  'Winnie Monsod',
  'Sonny Africa',
  'Margarito Teves'],
 '- Risa Hontiveros\n- Rosalina De Leon\n- Benjamin Diokno\n- Chiz 

In [94]:
contains_keys = [any(entity in ent_err_fmt.keys() for entity in entities) for entities in business_df['unique_entities']]

def update_encoding_format(entities):
    return [ent_err_fmt.get(entity, entity) for entity in entities][0]

business_df.loc[contains_keys, 'unique_entities'] = business_df['unique_entities'][contains_keys].apply(update_encoding_format)

business_df.head()

,id,date,date_gmt,modified,modified_gmt,slug,status,type,link,featured_media,comment_status,ping_status,sticky,template,format,categories,tags,movement,article_type,tier,ab_test_titles,ab_test_featured_images,authorship,guid.rendered,title.rendered,content.rendered,content.protected,excerpt.rendered,excerpt.protected,meta.rappler-three-point-summary,meta.claim_author_type,meta.claim_author_name,meta.claim_reviewed,meta.review_rating,meta.schema_article_type,meta.ep_exclude_from_search,meta.is_sponsored,meta.sponsor,ab_tests.titles.started,ab_tests.titles.start_time,ab_tests.titles.end_time,ab_tests.titles.traffic_percentage,ab_tests.titles.variant_traffic_percentage,ab_tests.titles.paused,ab_tests.featured_images.started,ab_tests.featured_images.start_time,ab_tests.featured_images.end_time,ab_tests.featured_images.traffic_percentage,ab_tests.featured_images.variant_traffic_percentage,ab_tests.featured_images.paused,_links.self,_links.collection,_links.about,_links.replies,_links.version-history,_links.predecessor-version,_links.wp:featuredmedia,_links.wp:attachment,_links.wp:term,_links.curies,content.cleaned,unique_entities,author.name
26,2452896,2023-07-24 12:43:49,2023-07-24T04:43:49,2023-07-24T13:01:14,2023-07-24T05:01:14,list-flights-canceled-delayed-typhoon-egay-jul...,publish,post,https://www.rappler.com/business/list-flights-...,2243073,open,closed,False,NaN,standard,"[621, 624]","[2283, 2487, 2802]",[],[],[],[],[],[6936],urn:uuid:71f84171-debc-4aa0-b232-5166cd1de723,"LIST: Canceled flights due to Typhoon Egay, Ju...","\n<p>MANILA, Philippines – Various airports an...",False,<p>Bookmark this page to see the affected flig...,False,"['[""Typhoon', 'Egay', '(Doksuri)', 'prompts', ...",NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192529000,1692784529000,35,[],True,False,1690192529000,1692784529000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 6, 'href': 'https://www.rappler.com...","[{'id': 2453139, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re...","MANILA, Philippines - Various airports and ai...",[],Lance Spencer Yu
34,2452759,2023-07-24 10:44:42,2023-07-24T02:44:42,2023-07-24T10:44:46,2023-07-24T02:44:46,repower-energy-initial-public-offering-july-24...,publish,post,https://www.rappler.com/business/repower-energ...,2452801,open,closed,False,NaN,standard,"[621, 625, 626]","[2716, 2520, 2266]",[],[],[],[],[],[4952],urn:uuid:3f2e008c-7f8f-4853-ba0f-949344983015,Repower Energy goes public in tepid market,"\n<p>MANILA, Philippines – Repower Energy Deve...",False,<p>Repower Energy Development Corporation is j...,False,['[]'],NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192530000,1692784530000,35,[],True,False,1690192530000,1692784530000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 6, 'href': 'https://www.rappler.com...","[{'id': 2452843, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re...","MANILA, Philippines - Repower Energy Developm...","[Eric Peter Roxas, Rodrigo Duterte]",Ralf Rivas
111,2449638,2023-07-22 14:42:06,2023-07-22T06:42:06,2023-07-22T14:42:11,2023-07-22T06:42:11,coa-finds-anti-red-tape-authority-complaint-re...,publish,post,https://www.rappler.com/business/coa-finds-ant...,2017041,open,closed,False,NaN,standard,"[621, 622]","[73, 1964]",[],[],[],[],[],[4999],urn:uuid:c48afd24-cc1e-4234-868a

In [117]:
irr_ent = open('irr-entities.txt').readlines()
irr_ent = [ent.rstrip('\n') for ent in irr_ent]

In [118]:
business_df['unique_entities'] = business_df['unique_entities'].apply(lambda x: [ent for ent in x if ent not in irr_ent])

# for k, v in business_df['unique_entities'].explode().value_counts().to_dict().items():
#     print(k)

## standardizing format of entities

In [119]:
labels = open('labels.txt').readlines()
labels = [lab.rstrip('\n') for lab in labels]

for lab in labels:
    business_df['unique_entities'] = business_df['unique_entities'].apply(
        lambda x: [ent.replace(lab, '').strip() if labels else ent for ent in x]
    )

In [154]:
std_names = {'Juan Miguel "Migz" Zubiri': 'Migz Zubiri',
'Juan Miguel Zubiri': 'Migz Zubiri',
'Jay Art Tugade' : 'Jose Arturo Tugade',
'Jose Arturo "Jay Art" Tugade' : 'Jose Arturo Tugade',
'Ferdinand Marcos Jr.': 'Bongbong Marcos',
'Ferdinand E. Marcos': 'Bongbong Marcos',
'BBM': 'Bongbong Marcos',
'Lucio "Hun Hun" Tan III': 'Lucio Tan III',
'Manny V. Pangilinan': 'Manny Pangilinan',
'Manuel V. Pangilinan': 'Manny Pangilinan',
'Manuel Pangilinan': 'Manny Pangilinan',
'Noli "Kabayan" de Castro': 'Noli de Castro',
'Manolito "Mannix" A. Manalo': 'Manolito Manalo',
'Sofonias "Ponyong" Gabonada Jr.': 'Sofonias Gabonada Jr.',
'DJ Toni Aquino': 'Toni Aquino',
'Cory Aquino': 'Corazon Aquino',
'Ninoy Aquino': 'Benigno "Ninoy" Aquino Jr.',
'TG Limcaoco':'Jose Teodoro "TG" Limcaoco'
}

joblib.dump(std_names, 'std-names.pkl')
# std_names = joblib.load('std-names.pkl')

['std-names.pkl']

In [155]:
contains_keys = [any(entity in std_names.keys() for entity in entities) for entities in business_df['unique_entities']]

def standardize_naming(entities):
    return [std_names.get(entity, entity) for entity in entities]

business_df.loc[contains_keys, 'unique_entities']  = business_df['unique_entities'][contains_keys].apply(standardize_naming)
business_df.head()

,id,date,date_gmt,modified,modified_gmt,slug,status,type,link,featured_media,comment_status,ping_status,sticky,template,format,categories,tags,movement,article_type,tier,ab_test_titles,ab_test_featured_images,authorship,guid.rendered,title.rendered,content.rendered,content.protected,excerpt.rendered,excerpt.protected,meta.rappler-three-point-summary,meta.claim_author_type,meta.claim_author_name,meta.claim_reviewed,meta.review_rating,meta.schema_article_type,meta.ep_exclude_from_search,meta.is_sponsored,meta.sponsor,ab_tests.titles.started,ab_tests.titles.start_time,ab_tests.titles.end_time,ab_tests.titles.traffic_percentage,ab_tests.titles.variant_traffic_percentage,ab_tests.titles.paused,ab_tests.featured_images.started,ab_tests.featured_images.start_time,ab_tests.featured_images.end_time,ab_tests.featured_images.traffic_percentage,ab_tests.featured_images.variant_traffic_percentage,ab_tests.featured_images.paused,_links.self,_links.collection,_links.about,_links.replies,_links.version-history,_links.predecessor-version,_links.wp:featuredmedia,_links.wp:attachment,_links.wp:term,_links.curies,content.cleaned,unique_entities,author.name
26,2452896,2023-07-24 12:43:49,2023-07-24T04:43:49,2023-07-24T13:01:14,2023-07-24T05:01:14,list-flights-canceled-delayed-typhoon-egay-jul...,publish,post,https://www.rappler.com/business/list-flights-...,2243073,open,closed,False,NaN,standard,"[621, 624]","[2283, 2487, 2802]",[],[],[],[],[],[6936],urn:uuid:71f84171-debc-4aa0-b232-5166cd1de723,"LIST: Canceled flights due to Typhoon Egay, Ju...","\n<p>MANILA, Philippines – Various airports an...",False,<p>Bookmark this page to see the affected flig...,False,"['[""Typhoon', 'Egay', '(Doksuri)', 'prompts', ...",NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192529000,1692784529000,35,[],True,False,1690192529000,1692784529000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 6, 'href': 'https://www.rappler.com...","[{'id': 2453139, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re...","MANILA, Philippines - Various airports and ai...",[],Lance Spencer Yu
34,2452759,2023-07-24 10:44:42,2023-07-24T02:44:42,2023-07-24T10:44:46,2023-07-24T02:44:46,repower-energy-initial-public-offering-july-24...,publish,post,https://www.rappler.com/business/repower-energ...,2452801,open,closed,False,NaN,standard,"[621, 625, 626]","[2716, 2520, 2266]",[],[],[],[],[],[4952],urn:uuid:3f2e008c-7f8f-4853-ba0f-949344983015,Repower Energy goes public in tepid market,"\n<p>MANILA, Philippines – Repower Energy Deve...",False,<p>Repower Energy Development Corporation is j...,False,['[]'],NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192530000,1692784530000,35,[],True,False,1690192530000,1692784530000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 6, 'href': 'https://www.rappler.com...","[{'id': 2452843, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re...","MANILA, Philippines - Repower Energy Developm...","[Eric Peter Roxas, Rodrigo Duterte]",Ralf Rivas
111,2449638,2023-07-22 14:42:06,2023-07-22T06:42:06,2023-07-22T14:42:11,2023-07-22T06:42:11,coa-finds-anti-red-tape-authority-complaint-re...,publish,post,https://www.rappler.com/business/coa-finds-ant...,2017041,open,closed,False,NaN,standard,"[621, 622]","[73, 1964]",[],[],[],[],[],[4999],urn:uuid:c48afd24-cc1e-4234-868a

In [161]:
business_df['unique_entities'] = business_df['unique_entities'].apply(lambda x: [ent.replace('.','') if '.' in ent else ent for ent in x])
business_df.head()

,id,date,date_gmt,modified,modified_gmt,slug,status,type,link,featured_media,comment_status,ping_status,sticky,template,format,categories,tags,movement,article_type,tier,ab_test_titles,ab_test_featured_images,authorship,guid.rendered,title.rendered,content.rendered,content.protected,excerpt.rendered,excerpt.protected,meta.rappler-three-point-summary,meta.claim_author_type,meta.claim_author_name,meta.claim_reviewed,meta.review_rating,meta.schema_article_type,meta.ep_exclude_from_search,meta.is_sponsored,meta.sponsor,ab_tests.titles.started,ab_tests.titles.start_time,ab_tests.titles.end_time,ab_tests.titles.traffic_percentage,ab_tests.titles.variant_traffic_percentage,ab_tests.titles.paused,ab_tests.featured_images.started,ab_tests.featured_images.start_time,ab_tests.featured_images.end_time,ab_tests.featured_images.traffic_percentage,ab_tests.featured_images.variant_traffic_percentage,ab_tests.featured_images.paused,_links.self,_links.collection,_links.about,_links.replies,_links.version-history,_links.predecessor-version,_links.wp:featuredmedia,_links.wp:attachment,_links.wp:term,_links.curies,content.cleaned,unique_entities,author.name
26,2452896,2023-07-24 12:43:49,2023-07-24T04:43:49,2023-07-24T13:01:14,2023-07-24T05:01:14,list-flights-canceled-delayed-typhoon-egay-jul...,publish,post,https://www.rappler.com/business/list-flights-...,2243073,open,closed,False,NaN,standard,"[621, 624]","[2283, 2487, 2802]",[],[],[],[],[],[6936],urn:uuid:71f84171-debc-4aa0-b232-5166cd1de723,"LIST: Canceled flights due to Typhoon Egay, Ju...","\n<p>MANILA, Philippines – Various airports an...",False,<p>Bookmark this page to see the affected flig...,False,"['[""Typhoon', 'Egay', '(Doksuri)', 'prompts', ...",NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192529000,1692784529000,35,[],True,False,1690192529000,1692784529000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 6, 'href': 'https://www.rappler.com...","[{'id': 2453139, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re...","MANILA, Philippines - Various airports and ai...",[],Lance Spencer Yu
34,2452759,2023-07-24 10:44:42,2023-07-24T02:44:42,2023-07-24T10:44:46,2023-07-24T02:44:46,repower-energy-initial-public-offering-july-24...,publish,post,https://www.rappler.com/business/repower-energ...,2452801,open,closed,False,NaN,standard,"[621, 625, 626]","[2716, 2520, 2266]",[],[],[],[],[],[4952],urn:uuid:3f2e008c-7f8f-4853-ba0f-949344983015,Repower Energy goes public in tepid market,"\n<p>MANILA, Philippines – Repower Energy Deve...",False,<p>Repower Energy Development Corporation is j...,False,['[]'],NaN,NaN,NaN,NaN,NaN,False,False,0,False,1690192530000,1692784530000,35,[],True,False,1690192530000,1692784530000,35,[],True,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'embeddable': True, 'href': 'https://www.rap...","[{'count': 6, 'href': 'https://www.rappler.com...","[{'id': 2452843, 'href': 'https://www.rappler....","[{'embeddable': True, 'href': 'https://www.rap...",[{'href': 'https://www.rappler.com/wp-json/wp/...,"[{'taxonomy': 'category', 'embeddable': True, ...","[{'name': 'wp', 'href': 'https://api.w.org/{re...","MANILA, Philippines - Repower Energy Developm...","[Eric Peter Roxas, Rodrigo Duterte]",Ralf Rivas
111,2449638,2023-07-22 14:42:06,2023-07-22T06:42:06,2023-07-22T14:42:11,2023-07-22T06:42:11,coa-finds-anti-red-tape-authority-complaint-re...,publish,post,https://www.rappler.com/business/coa-finds-ant...,2017041,open,closed,False,NaN,standard,"[621, 622]","[73, 1964]",[],[],[],[],[],[4999],urn:uuid:c48afd24-cc1e-4234-868a

In [162]:
business_df['unique_entities'].explode().dropna()

34         Eric Peter Roxas
34          Rodrigo Duterte
127          Edgar Galvante
127         Rodrigo Duterte
189      Edgar Injap Sia II
                ...        
12350          Cesar Chiong
12355        Jaime Bautista
12355       Bongbong Marcos
12355         Manuel Tamayo
12355          Cesar Chiong
Name: unique_entities, Length: 3678, dtype: object

In [163]:
business_df['unique_entities'].explode().dropna().nunique()

2154

In [167]:
final = business_df[business_df['unique_entities'].apply(len)>=2].copy()

In [169]:
joblib.dump(final, 'business.pkl')

['business.pkl']

In [170]:
final.to_csv('business.csv', index=False)